In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/brettnaul/miniconda3/envs/deep/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [5]:
from keras import backend as K
K.set_learning_phase(0)
%run asas_full.py 96 2 0.25 --no_train --model_type conv --lr 5e-4 --sim_type asas_full/n200 --nb_epoch 1 --n_min 200 --n_max 200 --m_max 32 --embedding 32 --filter_length 7 --batch_norm

/Users/brettnaul/Dropbox/Documents/timeflow/asas_full.py:66: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  for f in filenames]
/Users/brettnaul/miniconda3/envs/deep/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/brettnaul/miniconda3/envs/deep/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


[]
/Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/asas_full/n200/conv_096_x2_5m04_drop25_bn_f7_emb32
Loading /Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/asas_full/n200/conv_096_x2_5m04_drop25_bn_f7_emb32/weights.h5...


# Visualizing predictions

In [3]:
%%time
pred = model.predict({'main_input': X, 'aux_input': np.delete(X, 1, axis=2)}, batch_size=500)

NameError: name 'model' is not defined

In [ ]:
i = -1

In [ ]:
#inds = ~np.isnan(X_raw[i, :, 0])
inds = np.arange(args.n_min)
i += 1
#i = np.random.choice(inds)
t = np.cumsum(X_raw[i, inds, 0])
m = X[i, inds, 1]
#m = np.random.normal(size=len(t))
e = X_raw[i, inds, 2]

plt.errorbar(t, m, e, None, 'o');
#plt.plot(t, pred[i], 'o');
plt.errorbar(t, pred[i], None, None, 'o');

#from gatspy.periodic import LombScargleFast
#gat_resids = np.zeros(len(X))
#model_gat = LombScargleFast(fit_period=True, optimizer_kwds={'period_range': (0.05 * (t.max() - t.min()), 0.95 * (t.max() - t.min())), 'quiet': True}, silence_warnings=True)
#model_gat.fit(t, m)
#print(model_gat.score(model_gat.best_period).item())

#w_r, A_r, phi_r, b_r = pred_gru[i]
#x_r = A_r * np.sin(2 * np.pi * w_r * t + phi_r) + b_r
#plt.plot(t, x_r, '--')

plt.legend(['Original', 'Reconstructed'])
plt.title("MSE: {}, Var: {}".format((np.mean((m - np.squeeze(pred[i])) ** 2)), np.var(m)));

# Visualizing training progress

# New data

In [15]:
X_raw[:, :, 1].min()

5.3689999999999998

In [6]:
import cesium.datasets
from keras.utils.np_utils import to_categorical

# Discard samples from classes w/ insufficient examples
top_classes = ['Classical_Cepheid', 'Semireg_PV', 'Mira', 'RR_Lyrae_FM', 'W_Ursae_Maj', 'LSP', 'RSG']

data = cesium.datasets.fetch_asas_training()
top_class_inds = np.in1d(data['classes'], top_classes)
X_list = [np.c_[t, m, e] for t, m, e in zip(data['times'], data['measurements'], data['errors'])]
X_list = [X_list[i] for i in np.where(top_class_inds)[0]]
sub_array_list = [np.array_split(x, np.arange(args.n_max, len(x), step=args.n_max)) for x in X_list]
sub_array_list = [[el for el in x if len(el) >= args.n_min] for x in sub_array_list]
X_list = [el for x in sub_array_list for el in x]

classnames, indices = np.unique(data['classes'][top_class_inds], return_inverse=True)
y = np.repeat(indices, [len(x) for x in sub_array_list])
top_classes = [5, 25, 10, 16, 26,  9, 18]
top_class_inds = np.in1d(y, top_classes)
Y = to_categorical(y, len(top_classes))

X_raw = pad_sequences(X_list, value=np.nan, dtype='float', padding='post')
X, scale_params = preprocess(X_raw, args.m_max)

# Remove errors
X = X[:, :, :2]

Loaded data from cached archive.


In [7]:
encode_layer = [l for l in model.layers if l.name == 'encoding'][0]
inputs = [l for l in model.layers if 'Input' in str(l)]
encode = K.function([inputs[0].input, inputs[1].input], [encode_layer.output])
encoding, = encode([X, X[:, :, 0:1]])

In [8]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

enc_train, enc_test, y_train, y_test = train_test_split(encoding, y, test_size=0.25, random_state=42)

#asas_model = RandomForestClassifier(n_estimators=128)
asas_model = GridSearchCV(RandomForestClassifier(), {'n_estimators': [100, 250], 'criterion': ['gini', 'entropy'], 'max_features': [0.1, 0.2, 0.3]})
#asas_model = SVC()
asas_model.fit(enc_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'criterion': ['gini', 'entropy'], 'n_estimators': [100, 250], 'max_features': [0.1, 0.2, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [9]:
print("Training score: {}%".format(100 * asas_model.score(enc_train, y_train)))
print("Test score: {}%".format(100 * asas_model.score(enc_test, y_test)))

Training score: 100.0%
Test score: 56.69781931464174%
